In [4]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [16]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [17]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from s2p_on_o2 import util

In [18]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [19]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_local = '/media/rich/bigSSD/tmp_data/'
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

contents of    /media/rich/bigSSD/tmp_data/    will be copied to    /n/files/Neurobio/MICROSCOPE/Rich/data/test


In [20]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [30]:
# sftp.mkdir_safe(dir_data_remote)
sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

uploading /media/rich/bigSSD/tmp_data/day0_00001_00001.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00001.tif
uploading /media/rich/bigSSD/tmp_data/day0_00001_00002.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00002.tif


------------
# ==== Start ====

## Prepare username + password

In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [21]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [22]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...

Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer03 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-07-31

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Soci

In [23]:
## initialize ssh_compute
ssh_c = util.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on login05 which is a
8 core system with 15.50 GiB memory
running kernel 3.10.0 born on 2020-07-07

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Social 

## Prepare directories

In [272]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to
name_slurm = 'AEG21_14'

dir_data_MICROSCOPE = Path(r'/n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14').as_posix()
dir_data_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_14').as_posix()  ## contents of dir_data_local will be dumped into dir_data_remote

dir_fastDisk_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_fastDisk/AEG21/2022_05_14').as_posix()  ## to use for temporary storage


dir_S2pOutput_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14').as_posix()  ## outer directory. Final outputs will be in an inner directory
name_job = 'jobNum_'


# path_dispatcher_local = Path(r'C:\Users\scanimage\github_repos\s2p_on_o2').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_local = Path(r'/media/rich/Home_Linux_partition/github_repos/s2p_on_o2').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = (Path(dir_S2pOutput_remote) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server
path_s2pScript_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py').as_posix()  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server

print(f'contents of    {dir_data_MICROSCOPE}    will be copied to    {dir_data_remote}')

contents of    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14    will be copied to    /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_14


## Prepare commands

In [273]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir -p {dir_data_remote}",
    'copy_data': f"cp -r {dir_data_MICROSCOPE}/. {dir_data_remote}",
    'dispatch_s2p': f"python {str(path_dispatcher_remote)} {dir_S2pOutput_remote} {path_s2pScript_remote} {name_job} {dir_fastDisk_remote} {name_slurm} {dir_data_remote}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'make_dir': 'mkdir -p /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_14',
 'copy_data': 'cp -r /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/. /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_14',
 'dispatch_s2p': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14 /n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py jobNum_ /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_fastDisk/AEG21/2022_05_14 AEG21_14 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_14'}

## Upload `dispatcher.py` file

In [196]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Copy data from `MICROSCOPE` to `data1`

In [241]:
ssh_t.send_receive(commands['make_dir']);

ssh_t.send(commands['copy_data'])

pRAM/AEG21/2022_05_18
(base) [rh183@transfer03 ~]$ 


In [242]:
ssh_t.expect(str_success=f'[{username}', recv_timeout=0.3, total_timeout=60*15, sleep_time=0.1, verbose=True);

8G21/2022_05_18/. /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG21/2022_05_1 

(base) [rh183@transfer03 ~]$ 


## Dispatch `remote_run_s2p.py`

In [197]:
ssh_c.send(commands['dispatch_s2p']);

ssh_c.expect(str_success=prompt_snip);

meused,starttime,timelimit,reasonlist,minmemory,mincpus,gres
JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
53523653            AEG21_14            gpu_requeue         PENDING             0:00                N/A                 6:00:00             (QOSMinGRES)        240G                20                  N/A                 
53524600            AEG22_16            gpu_requeue         PENDING             0:00                2022-05-19T10:47:24 6:00:00             (Resources)         240G                20                  gres:gpu:rtx6000:1  
53524599            AEG22_14            gpu_requeue         RUNNING             3:34                2022-05-19T04:48:31 6:00:00             compute-gc-17-247   240G                20                  gres:gpu:rtx6000:1  
53524588            AEG22_13            gpu_requeue    

### check on job

In [206]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
ssh_c.expect(str_success=f"[{username}", total_timeout=1);

meused,starttime,timelimit,reasonlist,minmemory,mincpus,gres
JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
53524600            AEG22_16            gpu_requeue         PENDING             0:00                2022-05-19T10:47:24 6:00:00             (Resources)         240G                20                  gres:gpu:rtx6000:1  
53524599            AEG22_14            gpu_requeue         RUNNING             14:00               2022-05-19T04:48:31 6:00:00             compute-gc-17-247   240G                20                  gres:gpu:rtx6000:1  
53524588            AEG22_13            gpu_requeue         RUNNING             15:07               2022-05-19T04:47:24 6:00:00             compute-gc-17-246   240G                20                  gres:gpu:rtx6000:1  
53524567            AEG21_18            gpu_requeue    

In [319]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

2p_output/AEG21/2022_05_13/jobNum_0/print*

cat: /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_13/jobNum_0/print*: No such file or directory
(base) [rh183@login06 ~]$ 


In [27]:
ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

RUN COMPLETE!!!     Tue May 17 14:52:20 2022


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [274]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14    will be copied to    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output


In [275]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/. /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output'}

In [276]:
ssh_t.send_receive(commands['make_dir']);

e/AEG21/2022_05_14/suite2p_o2_output



In [277]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()